In [1]:
import tensorflow as tf
import os
from pathlib import Path

In [2]:
train_path = Path() / 'aclImdb' / 'train'
test_path = Path() / 'aclImdb' / 'test'

In [3]:
unsupervised_path = train_path / 'unsup'
if os.path.exists(unsupervised_path):
    for file in os.listdir(unsupervised_path):
        file_path = unsupervised_path / file
        os.remove(file_path)
    os.rmdir(unsupervised_path)
    print('Unsupervised dataset removed')
else:
    print('Unsupervised dataset does not exist')    

Unsupervised dataset does not exist


In [4]:
train_ds = tf.keras.utils.text_dataset_from_directory(
    train_path,
    batch_size=32,
    seed=42,
)

Found 25000 files belonging to 2 classes.


In [5]:
for i, label in enumerate(train_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to neg
Label 1 corresponds to pos


In [6]:
(test_ds, val_ds) = tf.keras.utils.text_dataset_from_directory(
    test_path,
    batch_size=32,
    validation_split=0.6,
    subset='both',
    seed=42,
)

Found 25000 files belonging to 2 classes.
Using 10000 files for training.
Using 15000 files for validation.


In [7]:
for i, label in enumerate(test_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to neg
Label 1 corresponds to pos


In [9]:
vocab_size = 10_000
embedding_dim = 16

In [20]:
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [21]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [23]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15)

Epoch 1/15
782/782 [==============================] - 16s 19ms/step - loss: 0.6430 - accuracy: 0.5585 - val_loss: 0.5151 - val_accuracy: 0.7205
Epoch 2/15
782/782 [==============================] - 8s 10ms/step - loss: 0.3930 - accuracy: 0.8260 - val_loss: 0.3466 - val_accuracy: 0.8366
Epoch 3/15
782/782 [==============================] - 8s 10ms/step - loss: 0.2914 - accuracy: 0.8810 - val_loss: 0.3131 - val_accuracy: 0.8513
Epoch 4/15
782/782 [==============================] - 8s 11ms/step - loss: 0.2480 - accuracy: 0.8988 - val_loss: 0.3155 - val_accuracy: 0.8385
Epoch 5/15
782/782 [==============================] - 8s 10ms/step - loss: 0.2229 - accuracy: 0.9124 - val_loss: 0.2901 - val_accuracy: 0.8713
Epoch 6/15
782/782 [==============================] - 8s 10ms/step - loss: 0.2019 - accuracy: 0.9212 - val_loss: 0.2862 - val_accuracy: 0.8805
Epoch 7/15
782/782 [==============================] - 8s 11ms/step - loss: 0.1845 - accuracy: 0.9290 - val_loss: 0.3047 - val_accuracy: 0.862